In [35]:
import pandas as pd

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

In [36]:
def format_time(x):
    if not any(i in x for i in ['DNF', 'DNS']):
        if ':' in x:
            return round(float(str(x).split(':')[1]) + (60 * float(str(x).split(':')[0])), 3) if x != 0 else 0
        else:
            return(round(float(x), 3))
    else:
        return x

In [37]:
def reverse_name(x):
    if str(x) == 'guanyu_zhou':
        name = x.split('_')
        return name[1] + '_' + name[0]
    else:
        return x

In [38]:
races = pd.read_csv(path+'data/races.csv')
qualifying = pd.read_csv(path+'data/qualifying.csv')
constructors = pd.read_csv(path+'data/constructor_standings.csv')
drivers = pd.read_csv(path+'data/driver_standings.csv')
results = pd.read_csv(path+'data/results.csv')
starting_grid = pd.read_csv(path+'data/starting-grid.csv')

In [39]:
results.query('grid == 0').shape

(50, 13)

In [40]:
df1 = pd.merge(races, results, how='inner', on=['season', 'round', 'circuit_id']).drop(['url','points', 'status', 'time', 'grid'], axis=1)
df2 = pd.merge(df1, drivers, how='left', on=['season', 'round', 'driver']) 
df3 = pd.merge(df2, constructors, how='left', on=['season', 'round', 'constructor'])
df4 = pd.merge(df3, qualifying, how='inner', on=['season', 'round', 'circuit_id', 'driver']).drop(['fastest_lap', 'fastest_time'], axis=1)
df4.rename(columns = {'grid': 'qualifying_pos'}, inplace=True)
df4.driver = df4.driver.apply(lambda x: reverse_name(x))

final_df = pd.merge(df4, starting_grid, how='inner', on=['season', 'round', 'driver'])
final_df.rename(columns = {'grid': 'starting_grid'}, inplace=True)

final_df.shape

(3651, 22)

In [41]:
final_df.query('season == 2021 & round == 15 & driver == "max_verstappen"')

,season,round,circuit_id,country,lat,long,date,driver,date_of_birth,nationality,...,driver_points,driver_wins,driver_standings_pos,constructor_points,constructor_wins,constructor_standings_pos,qualifying_pos,stage,q_delta,starting_grid
3058,2021,15,sochi,Russia,43.4057,39.9578,2021-09-26 12:00:00,max_verstappen,1997-09-30,Dutch,...,244.5,7,2,364.5,8,2,20,q1,0.0,20


In [42]:
final_df.columns

Index(['season', 'round', 'circuit_id', 'country', 'lat', 'long', 'date',
       'driver', 'date_of_birth', 'nationality', 'constructor', 'podium',
       'driver_points', 'driver_wins', 'driver_standings_pos',
       'constructor_points', 'constructor_wins', 'constructor_standings_pos',
       'qualifying_pos', 'stage', 'q_delta', 'starting_grid'],
      dtype='object')

In [43]:
null_cols = final_df.columns[final_df.isna().any()].tolist()

null_rows = final_df[final_df.isna().any(axis=1)]
null_rows = null_rows[['season', 'round', 'driver', 'podium', 'stage']]

null_rows.shape

(0, 5)

In [44]:
# fill/drop nulls
### Data is being lossed here

for col in ['driver_points', 'driver_wins', 'driver_standings_pos', 'constructor_points', 
            'constructor_wins', 'constructor_standings_pos']:
            
    final_df[col].fillna(0, inplace=True)
    final_df[col] = final_df[col].map(lambda x: int(x))
    
final_df.dropna(inplace=True)

final_df.shape

(3651, 22)

In [45]:
final_df.to_csv(path+'data/merged.csv', index=False)